In [1]:
#Import Libraries
import numpy as np
import pandas as pd

In [2]:
#Node
class NodeRegression():

    def __init__(
        self, 
        Y: list,
        X: pd.DataFrame,
        min_samples_split=None,
        max_depth=None,
        depth=None,
        node_type=None,
        rule=None):
      
        #Data goes to the node 
        self.Y = Y 
        self.X = X

        #Hyper parameters
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth


